<a href="https://colab.research.google.com/github/TekyaygilFethi/PythonSQLDataFrameConnector/blob/main/Python_PostgreSQL_Query_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODULE

In [1]:
import psycopg2
import psycopg2.extras

class PostgreSQLQueryCreator():
  def __init__(self):
    self.cursor = self.__create_connection()

  def __create_connection(self):
    conn = psycopg2.connect(
      database="", user='', password='', host='', port= '5432'
    ) 
    conn.autocommit = True

    return conn.cursor()

  @staticmethod
  def __create_sql_for_table_creation(table_name, df):
    create_table_sql = f"CREATE TABLE {table_name}( ID SERIAL, "

    df_col_types = {k:str(v[0]) for k,v in pd.DataFrame(df.dtypes).T.to_dict('list').items()}
    df_col_types

    for k,v in df_col_types.items():
      db_dtype = ""
      if v == "object" or v == "datetime64[ns]":
        db_dtype = "varchar(500)" 
      elif v == "int64":
        db_dtype = "integer"
      elif v == "float64":
        db_dtype = "float"

      create_table_sql += f"{k} {db_dtype},"

    create_table_sql = create_table_sql[:-1]
    create_table_sql += ")"

    return create_table_sql

  @staticmethod
  def __create_sql_for_table_insertion(table_name, df):
    sql = f"INSERT INTO {table_name} ({','.join(df.columns)}) VALUES({('%s, ' * len(df.columns))[:-2]})" ## we are using %s for prevent sql injection
    return sql

  def create_table(self, table_name, df):
    try:
      table_creation_sql = self.__create_sql_for_table_creation(table_name, df)
      self.cursor.execute(table_creation_sql)
      print("Table has been created!!")
    except Exception as msg:
      print("Table already exists!! Error:" + str(msg))

  def insert_into_table(self, table_name, df):
    table_insertion_sql = self.__create_sql_for_table_insertion(table_name, df)
    vals = [tuple(row[1]) for row in df.iterrows()]
    psycopg2.extras.execute_batch(self.cursor, table_insertion_sql, vals)
    print("All data has been inserted!")

  def create_and_insert_table(self, table_name, df):
    self.create_table(table_name, df)
    self.insert_into_table(table_name, df)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# EXAMPLE

In [2]:
import pandas as pd
# from SQLQueryCreator import PostgreSQLQueryCreator

In [3]:
d = {
    "Name": ["Obi-Wan Kenobi", "Anakin Skywalker", "Count Dooku", "Darth Maul"], 
    "LightsaberColor": ["Blue", "Blue", "Red", "Red"],
    "Age": [29, 19, 80, 32]
    }

hero_df = pd.DataFrame().from_dict(d)

In [4]:
hero_df

,Name,LightsaberColor,Age
0,Obi-Wan Kenobi,Blue,29
1,Anakin Skywalker,Blue,19
2,Count Dooku,Red,80
3,Darth Maul,Red,32


In [5]:
db_service = PostgreSQLQueryCreator()

In [6]:
db_service.create_table("herotable", hero_df)
db_service.insert_into_table("herotable", hero_df)

Table has been created!!
All data has been inserted!
